# Parse LC databases

This script tests on how to parse the databases for cams and images.



In [ ]:
import pandas

## Read Cam and LiDAR database


In [ ]:
#dataset_path = '/mnt/data/datasets/Spherical/test_training/'
dataset_path = '/tmp/training/'
lidar_ts_db_path = dataset_path + 'lidar_ts.csv'
cam_ts_db_path = dataset_path + 'cam_ts.csv'
missions_db_path = dataset_path + 'missions.csv'

print(f'Reading lidar db from {lidar_ts_db_path}')
print(f'Reading cam db from {cam_ts_db_path}')
print(f'Reading cam db from {missions_db_path}')

lidar_df = pandas.read_csv(lidar_ts_db_path, names=['mission_from', 'ts_from', 'mission_to', 'ts_to'], delimiter=';', header=None)
cam_df = pandas.read_csv(cam_ts_db_path, names=['mission_from', 'ts_from', 'mission_to', 'ts_to'], delimiter=';', header=None)
missions_df = pandas.read_csv(missions_db_path, names=['mission_anchor', 'mission_positive', 'mission_negative'], delimiter=',', comment='#', header=None)

print(f'\nRead {lidar_df.size} and {cam_df.size} entries.')
print(missions_df)

In [ ]:
training_missions = ['8d1b..0000', '25b1..0000', 'ef8b..0000', 'b03a..0000', '0167..0000', '472b..0000', '0282..0000', 'e2da..0000', '8a4a..0000', '657d..0000', 'f760..0000', '73cc..0000', '0569..0000', '174e..0000', 'b52f..0000']
test_missions = ['89de..0000', '96af..0000', 'd530..0000', 'd662..0000', '62d2..0000', '6fec..0000', 'd778..0000']

In [ ]:
training_indices_lidar = pandas.DataFrame()
training_indices_cam = pandas.DataFrame()

#for i in range(0, len(training_missions)):
for i in range(0, 1):
    print(f'Processing training mission: {training_missions[i]}')
    current_lidar_df = missions_df[missions_df['mission_anchor'] == training_missions[i]]    
    print(current_lidar_df['mission_positive'])
    mask = [False] * current_lidar_df['mission_positive'].size
    for j in range(0, len(training_missions)):
        mask = mask | current_lidar_df['mission_positive'].str.contains(training_missions[j]).values
    
    current_lidar_df = current_lidar_df[mask]        
    
    
    index_lidar_df = pandas.DataFrame({'index': current_lidar_df.index})    
    #print(f'lidar df size: {training_indices_lidar.size} and raw {index_lidar_df.size} and before {current_lidar_df.size}')
    training_indices_lidar = training_indices_lidar.append(index_lidar_df)
    #print(f'lidar after df size\n: {training_indices_lidar.size}')
    
    #current_cam_df = cam_df[cam_df['mission_from'] == training_missions[i]]
    #current_cam_df = current_cam_df[current_cam_df['mission_to'].isin(training_missions)]
    #index_cam_df = pandas.DataFrame({'index': current_cam_df.index})        
    #training_indices_cam = training_indices_cam.append(index_cam_df)
    

training_indices_lidar.drop_duplicates()
#training_indices_cam.drop_duplicates()

print(f'lidar df size: \n{training_indices_lidar}')
#print(f'cam df size: \n{training_indices_cam}')

#training_indices = training_indices_lidar.join(training_indices_cam.set_index('index'), on='index', how='inner')
#print(f'lidar df after size: {training_indices.size}')
#print(training_indices)

In [ ]:
test_indices_lidar = pandas.DataFrame()
test_indices_cam = pandas.DataFrame()
    
for i in range(0, len(test_missions)):
    print(f'Processing test mission: {test_missions[i]}')
    current_lidar_df = lidar_df[lidar_df['mission_from'] == test_missions[i]]
    current_lidar_df = current_lidar_df[current_lidar_df['mission_to'].isin(test_missions)]
    index_lidar_df = pandas.DataFrame({'index': current_lidar_df.index})    
    test_indices_lidar = test_indices_lidar.append(index_lidar_df)
    
    current_cam_df = cam_df[cam_df['mission_from'] == test_missions[i]]
    current_cam_df = current_cam_df[current_cam_df['mission_to'].isin(test_missions)]
    index_cam_df = pandas.DataFrame({'index': current_cam_df.index})        
    test_indices_cam = test_indices_cam.append(index_cam_df)
    
test_indices_lidar.drop_duplicates()
test_indices_cam.drop_duplicates()
test_indices = test_indices_lidar.join(test_indices_cam.set_index('index'), on='index', how='inner')
print(test_indices)

In [ ]:
cross_join = test_indices.join(training_indices.set_index('index'), on='index', how='inner')
assert(cross_join.size == 0)

